In [ ]:
!pip install gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.0/322.0 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 102.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import gradio as gr

# Load dataset
df = pd.read_csv('/content/lung_disease_dataset.csv')

# Encode categorical variables
label_encoders = {}
for col in ['Gender', 'Smoking_History', 'Pollution_Exposure', 'Cough', 'Shortness_of_Breath']:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Split data
X = df.drop(columns=['Lung_Disease'])
y = df['Lung_Disease']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build deep learning model
model = keras.Sequential([
    keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=32)

# Save model
model.save('/content/lung_disease_model.h5')

# Load model
def predict_lung_disease(*features):
    input_data = np.array(features).reshape(1, -1)
    input_data = scaler.transform(input_data)
    prediction = model.predict(input_data)[0][0]
    return '🛑 Disease Detected' if prediction > 0.5 else '✅ No Disease'

# Define Gradio Interface with Decorative & Interactive UI
description = """
<style>
    body { background-color: #121212; color: #ffffff; font-family: 'Arial', sans-serif; text-align: center; }
    h1 { color: #1DB954; font-size: 2.5em; }
    h3 { color: #BB86FC; font-size: 1.5em; }
    p { font-size: 1.2em; }
</style>

# 🫁 **Lung Disease Predictor**
### 🌡️ Enter your medical details and get an AI-powered prediction instantly!

🔹 **Accurate Deep Learning Model**
🔹 **Fast & Secure Predictions**
🔹 **User-Friendly Interactive Interface**

Enter the required medical parameters below and click **'Predict'** to analyze your lung health status.
"""

inputs = [gr.Number(label=col) for col in X.columns]
interface = gr.Interface(
    fn=predict_lung_disease,
    inputs=inputs,
    outputs='text',
    title='🔬 Lung Disease AI Predictor',
    description=description,
    theme='huggingface',
    live=True,
    allow_flagging='never',
    css="body { background-color: #1E1E1E; color: #FFF; } button { background-color: #BB86FC; color: #FFF; font-size: 1.2em; padding: 10px 20px; border-radius: 8px; }"
)

# Launch app
interface.launch(share=True)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.7675 - loss: 0.5235 - val_accuracy: 0.7745 - val_loss: 0.4669
Epoch 2/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7881 - loss: 0.4714 - val_accuracy: 0.7785 - val_loss: 0.4707
Epoch 3/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7772 - loss: 0.4809 - val_accuracy: 0.7810 - val_loss: 0.4615
Epoch 4/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7836 - loss: 0.4637 - val_accuracy: 0.7840 - val_loss: 0.4562
Epoch 5/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7851 - loss: 0.4666 - val_accuracy: 0.7825 - val_loss: 0.4534
Epoch 6/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7983 - loss: 0.4476 - val_accuracy: 0.7850 - val_loss: 0.4554
Epoch 7/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7886 - loss: 0.4622 - val_accuracy: 0.7835 - val_loss: 0.4521
Epoch 8/50
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7991 - loss: 0.4439 - val_accuracy: 0.

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/blocks.py:1096: UserWarning: Cannot load huggingface. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/huggingface (Request ID: Root=1-67bc1a96-66e865a40135de8a2397f825;f5be0bcd-60c4-498f-9cff-aa39c5316a48)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")
/usr/local/lib/python3.11/dist-packages/gradio/interface.py:403: Use

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4a74d11fe9431d8d68.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
